In [1]:
from dotenv import load_dotenv
import os
import pinecone
from pinecone import Pinecone as PineconeClient
from pinecone import ServerlessSpec
import pandas as pd
from openai import OpenAI

In [2]:
movies = [
  {
      "id": "movie1",
      "title": "7번방의 선물",
      "year": 2013,
      "genre": "드라마",
      "description": "억울한 누명을 쓰고 교도소에 수감된 아빠와 그의 어린 딸의 감동적인 스토리",
  },
  {
      "id": "movie2",
      "title": "미나리",
      "year": 2020,
      "genre": "드라마",
      "description": "한국계 미국인 가족의 따뜻하고 감성적인 성장 이야기",
  },
  {
      "id": "movie3",
      "title": "기생충",
      "year": 2019,
      "genre": "드라마",
      "description": "가난한 가족과 부자 가족 사이의 빈부격차를 그린 사회 풍자 드라마",
  },
  {
      "id": "movie4",
      "title": "범죄도시",
      "year": 2017,
      "genre": "범죄",
      "description": "형사가 범죄 조직을 소탕하는 범죄 액션 영화",
  },
  {
      "id": "movie5",
      "title": "범죄도시 2",
      "year": 2022,
      "genre": "범죄",
      "description": "형사와 범죄 조직의 대결을 그린 범죄 액션 영화의 속편",
  },
  {
      "id": "movie6",
      "title": "헤어질 결심",
      "year": 2022,
      "genre": "범죄",
      "description": "산에서 발생한 의문의 죽음(살인 사건)을 수사하던 형사가 피의자에게 이끌리며 벌어지는 미스터리 멜로 영화",
  },
  {
      "id": "movie7",
      "title": "다만 악에서 구하소서",
      "year": 2020,
      "genre": "범죄",
      "description": "청부 살인업자와 범죄 조직의 마지막 거래를 그린 범죄 액션 영화",
  },
]

In [3]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
  model="text-embedding-3-small",
  openai_api_key=os.getenv("OPENAI_API_KEY"),
)

C:\Users\user\AppData\Local\Temp\ipykernel_6096\2439254196.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


In [4]:
descriptions = [movie["description"] for movie in movies]
descriptions

['억울한 누명을 쓰고 교도소에 수감된 아빠와 그의 어린 딸의 감동적인 스토리',
 '한국계 미국인 가족의 따뜻하고 감성적인 성장 이야기',
 '가난한 가족과 부자 가족 사이의 빈부격차를 그린 사회 풍자 드라마',
 '형사가 범죄 조직을 소탕하는 범죄 액션 영화',
 '형사와 범죄 조직의 대결을 그린 범죄 액션 영화의 속편',
 '산에서 발생한 의문의 죽음(살인 사건)을 수사하던 형사가 피의자에게 이끌리며 벌어지는 미스터리 멜로 영화',
 '청부 살인업자와 범죄 조직의 마지막 거래를 그린 범죄 액션 영화']

In [5]:
movie_vectors = embeddings.embed_documents(descriptions)

In [6]:
movie_vectors[0][:5]

[0.0772140365224404,
 0.010441011297606051,
 -0.0856581367755727,
 0.023411448032334532,
 -0.0413076077416376]

In [8]:
pc = PineconeClient(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "movie-vector-index"
if not pc.has_index(index_name):
  pc.create_index(
    index_name, dimension=1536, spec=ServerlessSpec(cloud="aws", region="us-east-1")
  )
# 생성된 인덱스 연결
index = pc.Index(index_name)

c:\Users\user\Downloads\Data_A\prompt_basic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
vector_data = []
for movie, vector in zip(movies, movie_vectors):
  print(movie["title"], len(vector))
  meta = {
    "title": movie["title"],
    "year": movie["year"],
    "genre": movie["genre"],
    "description": movie["description"],
  }
  vector_data.append((movie["id"], vector, meta))

7번방의 선물 1536
미나리 1536
기생충 1536
범죄도시 1536
범죄도시 2 1536
헤어질 결심 1536
다만 악에서 구하소서 1536


In [10]:
index.upsert(vectors=vector_data)

{'upserted_count': 7}

# KNN 검색

In [11]:
query1 = "감성적인 드라마 영화 추천해줘."
query_vector1 = embeddings.embed_query(query1)
query_vector1[:5]

[0.011942672407744534,
 -0.015711813986873352,
 -0.11866442724554467,
 0.023948870327582927,
 0.03150832617301168]

In [13]:
result1 = index.query(vector=query_vector1, top_k=3, include_metadata=True)
for match in result1["matches"]:
  info = match["metadata"]
  print(f"제목: {info['title']} - 장르: {info['genre']}")

제목: 기생충 - 장르: 드라마
제목: 7번방의 선물 - 장르: 드라마
제목: 미나리 - 장르: 드라마


# 메타 데이터 기반 필터 검색

In [14]:
query2 = "영화" # 2020년 영화만 대상으로 검색
query_vector2 = embeddings.embed_query(query2)
filter_condition = {"year": {"$gte": 2020}}  #<gt <= gte lt lte
result2 = index.query(vector=query_vector2, top_k=3, filter=filter_condition, include_metadata=True)
for match in result2["matches"]:
  info = match["metadata"]
  print(f"제목: {info['title']} - 장르: {info['genre']} - 연도: {info['year']}")

제목: 범죄도시 2 - 장르: 범죄 - 연도: 2022.0
제목: 헤어질 결심 - 장르: 범죄 - 연도: 2022.0
제목: 다만 악에서 구하소서 - 장르: 범죄 - 연도: 2020.0


In [ ]:
""